In [1]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [2]:
# 데이터 로드
article_df = pd.read_csv("article.csv")
article_df.tail()

,title,link,content,category
1770,"카카오게임즈 달빛조각사, 서버 통합 실시",https://news.naver.com/main/read.nhn?mode=LSD&...,"(지디넷코리아=이도원 기자)카카오게임즈(각자 대표 남궁훈, 조계현)는 엑스엘게임즈...",105
1771,"벤츠코리아, ‘아우스빌둥’ 4기 공식 출범…독일식 일-학습 병행 프로그램",https://news.naver.com/main/read.nhn?mode=LSD&...,메르세데스-벤츠코리아 김지섭(왼쪽 두 번째) 대표이사 사장 직무대행 등이 7일 개최...,105
1772,코로나19 유전정보 분석하니…주요 변이 후보염기 27개 도출,https://news.naver.com/main/read.nhn?mode=LSD&...,미국 대확산기 변이 분석…스파이크 17개·RNA보호 10개 후보 찾아미국 확진자 및...,105
1773,"'빛의 계승자', 웹툰 '신의 탑' 콜라보 업데이트…'밤이 주인공으로?'",https://news.naver.com/main/read.nhn?mode=LSD&...,[엑스포츠뉴스 최지영 기자] 빛의 계승자가 웹툰 신의 탑과 콜라보를 진행했다.8일 ...,105
1774,‘땡큐 中 신장 공안’…인권 문제로 비난받는 디즈니 <뮬란>,https://news.naver.com/main/read.nhn?mode=LSD&...,디즈니 영화 <뮬란>이 중국의 인권탄압에 눈감았다는 비난에 직면했다. 신장 위구르족...,105


In [3]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, Y_train, Y_test = train_test_split(
    article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(Y_train), len(Y_test)

(1597, 178, 1597, 178)

In [4]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [5]:
# 학습
model = clf.fit(X_train.values.astype("str"), Y_train)

In [6]:
# 모델 성능 확인
Y_pred = model.predict(X_test)

In [7]:
list(Y_test[:5]), list(Y_pred[:5])

([104, 105, 101, 100, 105], [102, 105, 101, 100, 105])

In [8]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         100       0.73      0.83      0.77        29
         101       0.58      0.56      0.57        34
         102       0.68      0.58      0.62        26
         103       0.66      0.73      0.69        26
         104       0.97      0.82      0.89        39
         105       0.71      0.83      0.77        24

    accuracy                           0.72       178
   macro avg       0.72      0.72      0.72       178
weighted avg       0.73      0.72      0.73       178



#### 모델 사용하기

In [9]:
categories = {
    100 : "정치",
    101 : "경제",
    102 : "사회",
    103 : "생활/문화",
    104 : "세계",
    105 : "IT/과학",
}

In [10]:
contents = [
    "네이버와 카카오는 드론 기술 발전에 주력",
    "요즘 환율 주가 예측이 불가",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
]

In [11]:
model.predict(contents)

array([105, 101, 100], dtype=int64)

In [12]:
data = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(data)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,요즘 환율 주가 예측이 불가,101
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",100


In [13]:
round(max(model.predict_proba([contents[0]])[0]),2)

0.57

In [14]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(lambda data: round(max(model.predict_proba([data])[0]),2))
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학,0.57
1,요즘 환율 주가 예측이 불가,101,경제,0.29
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",100,정치,0.20


In [17]:
# 모델 저장
pickle.dump(model, open("clf.pkl", "wb"))